In [1]:
#inspect dataset

In [2]:
!pip install geopandas
!pip install shapely

In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.validation import make_valid
import numpy as np
import matplotlib.pyplot as plt

In [49]:
cond_prevn = pd.read_csv("500_Cities__Census_Tract-level_Data__GIS_Friendly_Format___2019_release_20250405.csv")
cond_prevn = cond_prevn.rename(columns={'TractFIPS': 'Geo_FIPS'})
cond_prevn = cond_prevn.rename(columns=cond_prevn.iloc[0]).drop(cond_prevn.index[0])
#Removing unrelated prevalence columns (not comorbidities with Asthma)
def drop_columns_with_keyword(df, keyword):
    # Identify columns containing the keyword (95CI)
    cols_to_drop = [col for col in df.columns if keyword.lower() in col.lower()]
    # Drop them and return the updated DataFrame
    return df.drop(columns=cols_to_drop)
cond_prevn = drop_columns_with_keyword(cond_prevn,'95ci')
cond_prevn = cond_prevn.drop(['ARTHRITIS_CrudePrev','BINGE_CrudePrev','CANCER_CrudePrev','COLON_SCREEN_CrudePrev',
                              'DENTAL_CrudePrev','KIDNEY_CrudePrev','LPA_CrudePrev',
                              'MAMMOUSE_CrudePrev', 'MHLTH_CrudePrev','PAPTEST_CrudePrev',
                              'PHLTH_CrudePrev','SLEEP_CrudePrev','STROKE_CrudePrev','TEETHLOST_CrudePrev'], axis=1, errors='ignore')
cond_prevn['lat'] = cond_prevn['Geolocation'].str[1:15]
cond_prevn['lon'] = cond_prevn['Geolocation'].str[-15:-1]
cond_prevn = cond_prevn.drop(['Place_TractID','Geolocation'], axis=1)
cond_prevn

,StateAbbr,PlaceName,PlaceFIPS,TractFIPS,Population2010,ACCESS2_CrudePrev,BPHIGH_CrudePrev,BPMED_CrudePrev,CASTHMA_CrudePrev,CHD_CrudePrev,...,CHOLSCREEN_CrudePrev,COPD_CrudePrev,COREM_CrudePrev,COREW_CrudePrev,CSMOKING_CrudePrev,DIABETES_CrudePrev,HIGHCHOL_CrudePrev,OBESITY_CrudePrev,lat,lon
1,PA,Allentown,4202000,42077000101,3661,12.4,28.6,74.2,10,6,...,77.7,6.3,38.6,33.1,21.1,9.9,30.7,33.4,40.62928770640,-75.4306548629
2,PA,Allentown,4202000,42077000102,4150,14.8,33.1,76.5,10.7,7.8,...,77.3,8.6,33.9,28.1,24.9,12.7,34.2,37.2,40.62310078700,-75.4455884082
3,PA,Allentown,4202000,42077000400,3548,20.5,31.4,73.4,11.3,7.6,...,72,8.9,30,25.3,28,12.9,32.6,39.6,40.61686565200,-75.4614158392
4,PA,Allentown,4202000,42077000500,2646,25.5,32.8,72,11,7.2,...,70.7,8.1,24.2,20.9,29.1,15.7,32.4,45,40.60540785140,-75.4582342428
5,PA,Allentown,4202000,42077000600,6243,17.8,31.3,74.4,10.5,6.9,...,75.6,7.9,33.3,28.3,25.1,12.4,32.6,38.5,40.59240545150,-75.4494513428
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627,PA,Scranton,4269000,42069102700,2747,10.5,31.6,75.7,10.2,6.9,...,76.9,7.7,39.6,32.9,23.5,9.8,34.2,32,41.39375309740,-75.6565294528
628,PA,Scranton,4269000,42069102800,3060,7.6,36.5,80.8,9.5,8.3,...,83.3,7.9,44.1,36.3,18.5,11.1,38,30.7,41.36516563930,-75.6493931804
629,PA,Scranton,4269000,42069102900,4160,15.3,36.9,77.9,11.3,9.5,...,74.4,10.7,31.9,23.5,27.1,13.5,36.7,38.5,41.38146756020,-75.6741915785
630,PA,Scranton,4269000,42069103000,2141,13.4,33.9,76.3,10.5,7.9,...,76.5,8.7,36.4,29.6,24.8,11.6,35.5,34.9,41.38873828210,-75.6808207737


In [50]:
cond_prevn.shape

(631, 21)

In [51]:
ER_use = pd.read_csv("Allegheny_county_ER_use.csv")
ER_use.head()

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,YearOfStartDate,UnderAge1PopEst2015_19ACS,Age0to17PopEst2015_19ACS,...,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,420031301002,42,3,130100,2,518,2016.0,9.4,207,...,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,2,420031301002,42,3,130100,2,518,2017.0,9.4,207,...,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,3,420031301002,42,3,130100,2,518,2018.0,9.4,207,...,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,4,420031301002,42,3,130100,2,518,2019.0,9.4,207,...,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,5,420031301003,42,3,130100,3,489,2016.0,0.0,134,...,53.0,39.0,29.1,13.0,9.7,12.0,2.0,1.5,10.0,7.5


In [14]:
ER_use.shape

(4370, 23)

In [15]:
PC_use = pd.read_csv("Allegheny_county_PC_use.csv")
PC_use.head()

,_id,Geo_FIPS,StateFIPS,CountyFIPS,TractFIPS,BlockgroupFIPS,TotalPopEst2015_19ACS,Age0to17PopEst2015_19ACS,YearOfContactDate,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,WellChildVisitsInPastYearPer100PrimaryCarePatients,AsthmaDiagnosisAge0to17,AsthmaDiagnosisPer100PrimaryCarePatientsAge0to17
0,1,420031301002,42,3,130100,2,518,207,2016,82,59,72.0,17.0,20.7
1,2,420031301002,42,3,130100,2,518,207,2017,94,58,61.7,19.0,20.2
2,3,420031301002,42,3,130100,2,518,207,2018,89,63,70.8,19.0,21.3
3,4,420031301002,42,3,130100,2,518,207,2019,86,61,70.9,17.0,19.8
4,5,420031301003,42,3,130100,3,489,134,2016,53,34,64.2,18.0,34.0


In [16]:
PC_use.shape

(4366, 14)

In [17]:
#merging utilization data for Asthma ER and primary care
util_data = pd.merge(PC_use,ER_use, on="Geo_FIPS", how="inner" )
#dropping repeat columns
util_data = util_data.drop(["_id_y","StateFIPS_y","CountyFIPS_y","TractFIPS_y","BlockgroupFIPS_y",
                            "TotalPopEst2015_19ACS_y","Age0to17PopEst2015_19ACS_y","YearOfContactDate"],axis="columns")
#preparing Geo_FIPS for future join on census data
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].apply(str)
util_data['Geo_FIPS'] = util_data['Geo_FIPS'].str[:-1]
#grouping data on GEO_FIPS to facilitate merge on cebsus data
util_data
#util_data_grouped = util_data.groupby('Geo_FIPS').mean()
#util_data_grouped

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,...,ED_VisitsPer100Age0to17,NumberLowAcuityED_VisitsAge0to17,LowAcuityED_VisitsPer100Age0to17,NumberAsthmaRelatedED_Visits,AsthmaRelatedED_VisitsPer100Age0to17,NumberED_VisitsByChildrenUnder1YearOld,NumberOfInjuryRelatedVisits,InjuryRelatedEDVisitsPer100Age0to17,NumberAcuteRespiratoryTractInfectionRelatedED_VisitsAge0to17,AcuteRespiratoryTractInfectionRelatedED_VisitsPer100Age0to17
0,1,42003130100,42,3,130100,2,518,207,82,59,...,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
1,1,42003130100,42,3,130100,2,518,207,82,59,...,62.8,80.0,38.6,54.0,26.1,11.0,1.0,0.5,27.0,13.0
2,1,42003130100,42,3,130100,2,518,207,82,59,...,66.7,85.0,41.1,59.0,28.5,13.0,2.0,1.0,19.0,9.2
3,1,42003130100,42,3,130100,2,518,207,82,59,...,71.5,96.0,46.4,68.0,32.9,7.0,NaN,0.0,27.0,13.0
4,2,42003130100,42,3,130100,2,518,207,94,58,...,58.9,62.0,30.0,40.0,19.3,14.0,1.0,0.5,18.0,8.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16364,4365,42003982200,42,3,982200,1,4618,80,19,14,...,16.3,8.0,10.0,1.0,1.3,4.0,NaN,0.0,NaN,0.0
16365,4366,42003982200,42,3,982200,1,4618,80,16,8,...,12.5,2.0,2.5,NaN,0.0,1.0,NaN,0.0,NaN,0.0
16366,4366,42003982200,42,3,982200,1,4618,80,16,8,...,12.5,2.0,2.5,1.0,1.3,1.0,NaN,0.0,3.0,3.8
16367,4366,42003982200,42,3,982200,1,4618,80,16,8,...,11.3,2.0,2.5,NaN,0.0,2.0,NaN,0.0,3.0,3.8


In [20]:
#loading racial data from 2010 census
Race_data = pd.read_csv("2010_census_race_P9-Data.csv")
#removing extra details on multi-racial individuals
Race_data = Race_data.iloc[:,:13]
Race_data.head()

,Geography,Geographic Area Name,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,Total!!Not Hispanic or Latino!!Population of one race!!Asian alone,Total!!Not Hispanic or Latino!!Population of one race!!Native Hawaiian and Other Pacific Islander alone,Total!!Not Hispanic or Latino!!Population of one race!!Some Other Race alone,Total!!Not Hispanic or Latino!!Two or More Races
0,0500000US42003,"Allegheny County, Pennsylvania",1223348,19070,1204278,1183929,986212,159998,1426,33944,251,2098,20349
1,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",6600,161,6439,6362,4260,2004,12,72,2,12,77
2,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",3629,119,3510,3422,2642,573,8,171,1,27,88
3,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",616,10,606,595,459,96,1,39,0,0,11
4,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",2256,32,2224,2179,298,1846,7,24,0,4,45


In [21]:
Race_data.shape

(1503, 13)

In [ ]:
#loading 5 year HH income data adj to 2016 inflation levels
HHincome_data = pd.read_csv("/2016_ad_HHincome.S2503-Data.csv")
#removed data on Housing cost as % of hh income
HHincome_data = HHincome_data.iloc[:,:-133]
HHincome_data.head()

,Geography,Geographic Area Name,Occupied housing units!!Estimate!!Occupied housing units,Occupied housing units!!Margin of Error!!Occupied housing units,Owner-occupied housing units!!Estimate!!Occupied housing units,Owner-occupied housing units!!Margin of Error!!Occupied housing units,Renter-occupied housing units!!Estimate!!Occupied housing units,Renter-occupied housing units!!Margin of Error!!Occupied housing units,"Occupied housing units!!Estimate!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000","Occupied housing units!!Margin of Error!!HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2016 INFLATION-ADJUSTED DOLLARS)!!Less than $5,000",...,Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars)
0,0500000US42003,"Allegheny County, Pennsylvania",531075,2098,343625,2325,187450,2003,3.4,0.2,...,0.1,(X),(X),4.4,0.3,866,4,934,8,800
1,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",281,38,73,31,208,44,4.6,5.1,...,7.7,(X),(X),0,10.2,800,275,320,216,1000
2,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",2042,160,408,116,1634,154,4.8,2.5,...,1.1,(X),(X),0,1.4,1472,110,1395,188,1496
3,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",553,48,86,37,467,57,2.7,3,...,4,(X),(X),0,4.7,1747,93,1769,929,1809
4,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",1117,96,211,51,906,116,8.3,6.1,...,2,(X),(X),0,2.4,724,78,872,164,693


In [ ]:
#Merging census datasets
census_data = pd.merge(Race_data, HHincome_data, on='Geography', how='inner')
census_data = census_data.drop(index=0)
census_data['Geo_FIPS'] = census_data['Geography'].str[-11:]
col = census_data.pop('Geo_FIPS')
census_data.insert(0, 'Geo_FIPS', col)
census_data.head(5)

,Geo_FIPS,Geography,Geographic Area Name_x,Total,Total!!Hispanic or Latino,Total!!Not Hispanic or Latino,Total!!Not Hispanic or Latino!!Population of one race,Total!!Not Hispanic or Latino!!Population of one race!!White alone,Total!!Not Hispanic or Latino!!Population of one race!!Black or African American alone,Total!!Not Hispanic or Latino!!Population of one race!!American Indian and Alaska Native alone,...,Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars)
1,42003010300,1400000US42003010300,"Census Tract 103, Allegheny County, Pennsylvania",6600,161,6439,6362,4260,2004,12,...,7.7,(X),(X),0,10.2,800,275,320,216,1000
2,42003020100,1400000US42003020100,"Census Tract 201, Allegheny County, Pennsylvania",3629,119,3510,3422,2642,573,8,...,1.1,(X),(X),0,1.4,1472,110,1395,188,1496
3,42003020300,1400000US42003020300,"Census Tract 203, Allegheny County, Pennsylvania",616,10,606,595,459,96,1,...,4,(X),(X),0,4.7,1747,93,1769,929,1809
4,42003030500,1400000US42003030500,"Census Tract 305, Allegheny County, Pennsylvania",2256,32,2224,2179,298,1846,7,...,2,(X),(X),0,2.4,724,78,872,164,693
5,42003040200,1400000US42003040200,"Census Tract 402, Allegheny County, Pennsylvania",2604,61,2543,2505,1331,997,7,...,1.6,(X),(X),2.1,2.2,581,132,500,187,691


In [22]:
#Loading Emissions data
emissions_data = pd.read_csv("Emissions_Data_w_Census_Tracts.csv")
# Retaining data from 2016-2019 to align with utilization data
emissions_data = emissions_data[(emissions_data["year"] <= 2019) & (emissions_data["year"] >= 2016)]
emissions_data.head()

,OBJECTID,_id,year,pf_id,facility,pollutant,tons_per_yr,ghg,hap,criteria,...,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,LSAD,ALAND,AWATER,SHAPE_Leng
9863,9864,9864,2016,737247,Eurovia,Acetaldehyde,0.28,0,1,0,...,42,3,458000,1400000US42003458000,42003458000,4580.0,CT,35159919,123974,0.274069
9864,9865,9865,2016,737247,Eurovia,Benzene,0.34,0,1,0,...,42,3,458000,1400000US42003458000,42003458000,4580.0,CT,35159919,123974,0.274069
9865,9866,9866,2016,737247,Eurovia,Carbon Monoxide,8.83,0,0,1,...,42,3,458000,1400000US42003458000,42003458000,4580.0,CT,35159919,123974,0.274069
9866,9867,9867,2016,737247,Eurovia,Ethylbenzene,0.15,0,1,0,...,42,3,458000,1400000US42003458000,42003458000,4580.0,CT,35159919,123974,0.274069
9867,9868,9868,2016,737247,Eurovia,Formaldehyde,0.66,0,1,0,...,42,3,458000,1400000US42003458000,42003458000,4580.0,CT,35159919,123974,0.274069


In [23]:
emissions_data.shape

(4768, 28)

In [ ]:
#Merging health care utlization data with census data on race and household income
utilxcensus_data = pd.merge(util_data,census_data, on="Geo_FIPS", how="inner")
utilxcensus_data['Geo_FIPS']=utilxcensus_data['Geo_FIPS'].apply(int)
utilxcensus_data

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,...,Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!No cash rent,Renter-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!No cash rent,Occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars),Owner-occupied housing units!!Margin of Error!!MONTHLY HOUSING COSTS!!Median (dollars),Renter-occupied housing units!!Estimate!!MONTHLY HOUSING COSTS!!Median (dollars)
0,1,42003130100,42,3,130100,2,518,207,82,59,...,4.4,(X),(X),8.4,7.4,486,133,511,91,455
1,1,42003130100,42,3,130100,2,518,207,82,59,...,4.4,(X),(X),8.4,7.4,486,133,511,91,455
2,1,42003130100,42,3,130100,2,518,207,82,59,...,4.4,(X),(X),8.4,7.4,486,133,511,91,455
3,1,42003130100,42,3,130100,2,518,207,82,59,...,4.4,(X),(X),8.4,7.4,486,133,511,91,455
4,2,42003130100,42,3,130100,2,518,207,94,58,...,4.4,(X),(X),8.4,7.4,486,133,511,91,455
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16364,4365,42003982200,42,3,982200,1,4618,80,19,14,...,52.6,(X),(X),0,58.2,970,137,-,**,950
16365,4366,42003982200,42,3,982200,1,4618,80,16,8,...,52.6,(X),(X),0,58.2,970,137,-,**,950
16366,4366,42003982200,42,3,982200,1,4618,80,16,8,...,52.6,(X),(X),0,58.2,970,137,-,**,950
16367,4366,42003982200,42,3,982200,1,4618,80,16,8,...,52.6,(X),(X),0,58.2,970,137,-,**,950


In [ ]:
#merged census and utilization data with health condition prevalence data
semi_data = pd.merge(utilxcensus_data, cond_prevn, on="Geo_FIPS", how="inner")
semi_data['TRACTCE'] = semi_data['TractFIPS_x']
semi_data

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,...,DIABETES_Crude95CI,HIGHCHOL_CrudePrev,HIGHCHOL_Crude95CI,OBESITY_CrudePrev,OBESITY_Crude95CI,Geolocation,lat,lon,geometry,TRACTCE
0,1,42003130100,42,3,130100,2,518,207,82,59,...,"(21.7, 23.8)",36.6,"(35.7, 37.6)",49.3,"(48.2, 50.4)","(40.45925070180, -79.8886885520)",40.45925070180,-79.8886885520,POINT (-79.888688552 40.4592507018),130100
1,1,42003130100,42,3,130100,2,518,207,82,59,...,"(21.7, 23.8)",36.6,"(35.7, 37.6)",49.3,"(48.2, 50.4)","(40.45925070180, -79.8886885520)",40.45925070180,-79.8886885520,POINT (-79.888688552 40.4592507018),130100
2,1,42003130100,42,3,130100,2,518,207,82,59,...,"(21.7, 23.8)",36.6,"(35.7, 37.6)",49.3,"(48.2, 50.4)","(40.45925070180, -79.8886885520)",40.45925070180,-79.8886885520,POINT (-79.888688552 40.4592507018),130100
3,1,42003130100,42,3,130100,2,518,207,82,59,...,"(21.7, 23.8)",36.6,"(35.7, 37.6)",49.3,"(48.2, 50.4)","(40.45925070180, -79.8886885520)",40.45925070180,-79.8886885520,POINT (-79.888688552 40.4592507018),130100
4,2,42003130100,42,3,130100,2,518,207,94,58,...,"(21.7, 23.8)",36.6,"(35.7, 37.6)",49.3,"(48.2, 50.4)","(40.45925070180, -79.8886885520)",40.45925070180,-79.8886885520,POINT (-79.888688552 40.4592507018),130100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4368,4365,42003982200,42,3,982200,1,4618,80,19,14,...,"( 1.6, 2.1)",9.4,"( 8.5, 10.5)",25.5,"(23.0, 28.2)","(40.44448787190, -79.9547651976)",40.44448787190,-79.9547651976,POINT (-79.9547651976 40.4444878719),982200
4369,4366,42003982200,42,3,982200,1,4618,80,16,8,...,"( 1.6, 2.1)",9.4,"( 8.5, 10.5)",25.5,"(23.0, 28.2)","(40.44448787190, -79.9547651976)",40.44448787190,-79.9547651976,POINT (-79.9547651976 40.4444878719),982200
4370,4366,42003982200,42,3,982200,1,4618,80,16,8,...,"( 1.6, 2.1)",9.4,"( 8.5, 10.5)",25.5,"(23.0, 28.2)","(40.44448787190, -79.9547651976)",40.44448787190,-79.9547651976,POINT (-79.9547651976 40.4444878719),982200
4371,4366,42003982200,42,3,982200,1,4618,80,16,8,...,"( 1.6, 2.1)",9.4,"( 8.5, 10.5)",25.5,"(23.0, 28.2)","(40.44448787190, -79.9547651976)",40.44448787190,-79.9547651976,POINT (-79.9547651976 40.4444878719),982200


In [ ]:
#joining emissions dataset
data = pd.merge(semi_data, emissions_data, on="TRACTCE", how="inner")
#dropping duplicate columns
data = data.drop(['TARGET_FID','STATEFP','COUNTYFP','TRACTCE','AFFGEOID','GEOID','NAME'],axis=1)
data

,_id_x,Geo_FIPS,StateFIPS_x,CountyFIPS_x,TractFIPS_x,BlockgroupFIPS_x,TotalPopEst2015_19ACS_x,Age0to17PopEst2015_19ACS_x,AllPrimaryCarePatientsAge0to17,WellChildVisitsInPastYearAge0to17,...,current,lat_y,lon_y,OBJECTID.1,Join_Count,FID_1,LSAD,ALAND,AWATER,SHAPE_Leng
0,101,42003140500,42,3,140500,1,589,61,54,35,...,0,40.448250,-79.893731,10639,1,288,CT,754718,0,0.046513
1,101,42003140500,42,3,140500,1,589,61,54,35,...,0,40.448250,-79.893731,10640,1,288,CT,754718,0,0.046513
2,101,42003140500,42,3,140500,1,589,61,54,35,...,0,40.448250,-79.893731,10641,1,288,CT,754718,0,0.046513
3,101,42003140500,42,3,140500,1,589,61,54,35,...,0,40.448250,-79.893731,10642,1,288,CT,754718,0,0.046513
4,101,42003140500,42,3,140500,1,589,61,54,35,...,0,40.448250,-79.893731,10643,1,288,CT,754718,0,0.046513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20667,4366,42003982200,42,3,982200,1,4618,80,16,8,...,1,40.442127,-79.949130,14469,1,214,CT,505672,0,0.047812
20668,4366,42003982200,42,3,982200,1,4618,80,16,8,...,1,40.442127,-79.949130,14470,1,214,CT,505672,0,0.047812
20669,4366,42003982200,42,3,982200,1,4618,80,16,8,...,1,40.442127,-79.949130,14471,1,214,CT,505672,0,0.047812
20670,4366,42003982200,42,3,982200,1,4618,80,16,8,...,1,40.442127,-79.949130,14472,1,214,CT,505672,0,0.047812
